In [1]:
from sklearn import datasets
iris=datasets.load_iris()
#converting continuous data into categorical data by dividing continuous data in various classes
for i in range(0,4):
    mean1=iris.data[:,i].mean()
    max1=iris.data[:,i].max()
    min1=iris.data[:,i].min()
    
    for j in range(0,150):
       

        if iris.data[j,i]<=min1+(mean1-min1)/2:
            iris.data[j,i]=min1
        elif iris.data[j,i]>min1+(mean1-min1)/2 and iris.data[j,i]<mean1:
            iris.data[j,i]=min1+ (mean1-min1)/2
        elif iris.data[j,i]>=mean1 and iris.data[j,i]<mean1+(max1-mean1)/2:
            iris.data[j,i]=mean1
        elif iris.data[j,i]>=mean1+(max1-mean1)/2 and iris.data[j,i]<=max1:
            iris.data[j,i]=mean1+(max1-mean1)/2  

iris.data 

array([[ 5.07166667,  3.054     ,  1.        ,  0.1       ],
       [ 4.3       ,  2.527     ,  1.        ,  0.1       ],
       [ 4.3       ,  3.054     ,  1.        ,  0.1       ],
       [ 4.3       ,  3.054     ,  1.        ,  0.1       ],
       [ 4.3       ,  3.054     ,  1.        ,  0.1       ],
       [ 5.07166667,  3.727     ,  1.        ,  0.1       ],
       [ 4.3       ,  3.054     ,  1.        ,  0.1       ],
       [ 4.3       ,  3.054     ,  1.        ,  0.1       ],
       [ 4.3       ,  2.527     ,  1.        ,  0.1       ],
       [ 4.3       ,  3.054     ,  1.        ,  0.1       ],
       [ 5.07166667,  3.054     ,  1.        ,  0.1       ],
       [ 4.3       ,  3.054     ,  1.        ,  0.1       ],
       [ 4.3       ,  2.527     ,  1.        ,  0.1       ],
       [ 4.3       ,  2.527     ,  1.        ,  0.1       ],
       [ 5.07166667,  3.727     ,  1.        ,  0.1       ],
       [ 5.07166667,  3.727     ,  1.        ,  0.1       ],
       [ 5.07166667,  3.

In [2]:
import numpy as np
def fit(x_train,y_train):
    result={}
    class_values= set(y_train)
    for current_class in class_values:
        result[current_class]={}
        result['total_data']=len(y_train)
        current_class_rows=(y_train==current_class) # doubt
        x_train_current=x_train[current_class_rows]
        y_train_current=y_train[current_class_rows]
        num_features=x_train.shape[1]
        result[current_class]['total_count']=len(y_train_current)
        for j in range(1,num_features+1):
            result[current_class][j]={}
            all_possible_values=set(x_train[:,j-1])
            for current_value in all_possible_values:
                result[current_class][j][current_value]=(x_train_current[:,j-1]==current_value).sum()
    return result            

In [3]:
def probability(dictionary,x,current_class):
    output=np.log(dictionary[current_class]['total_count']/dictionary['total_data'])
    num_features=len(dictionary[current_class].keys())-1
    for j in range(1,num_features +1):
        xj=x[j-1]
        count_current_class_with_value_xj=dictionary[current_class][j][xj] +1
        count_current_class= dictionary[current_class]['total_count'] + len(dictionary[current_class][j].keys())
        current_xj_probability=np.log(count_current_class_with_value_xj/count_current_class)
        output= output+current_xj_probability
    return output

In [4]:
def predictSinglePoint(dictionary,x):
    classes=dictionary.keys()
    best_p=-1000
    best_class=-1
    first_run=True
    for current_class in classes:
        if(current_class=='total_data'):
            continue
        p_current_class=probability(dictionary,x,current_class)
        if(first_run or p_current_class > best_p):
            best_p=p_current_class
            best_class=current_class
        first_run=False
    return best_class    

In [5]:
def predict(dictionary,x_test):
    y_pred=[]
    for x in x_test:
        x_class=predictSinglePoint(dictionary,x)
        y_pred.append(x_class)
    return y_pred    
    

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [8]:
x_train,x_test,y_train,y_test= train_test_split(iris.data,iris.target)
dictionary=fit(iris.data,iris.target)
y_train_pred=predict(dictionary,x_train)
y_test_pred=predict(dictionary,x_test)
confusion_matrix(y_test,y_test_pred)


array([[16,  0,  0],
       [ 0, 11,  0],
       [ 0,  0, 11]], dtype=int64)